In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

import warnings
warnings.filterwarnings("ignore")

Разберём стекинг на практике, проанализировав датасет, описывающий параметры, которые были сняты со спутника при фотографировании Земли. У нас есть 54 переменные. Для упрощения будем рассматривать два типа поверхностей —так мы сводим нашу задачу к задаче бинарной классификации. Сделаем базовую предобработку, воспользуемся StandardScaler.

Стекинг позволяет объединять ответы нескольких алгоритмов первого уровня в один большой ответ при помощи нового алгоритма обучения. Для избежания переобучения будем разбивать обучающую выбоку на фолды — разбиение обучающей выборки на несколько частей. На каждом фолде мы обучаем алгоритм заново.

Функция compute_meta_feature принимает на вход один алгоритм и возвращает новые признаки на объектах, которые не использовались во время обучения. Функция generate_meta_feature делает подобное, но принимает на вход несколько классификаторов в списке и повторяет процедуру, а затем генерирует и возвращает матрицу с изначальным количеством объектов, признаков будет столько, сколько мы передали классификаторов. 

In [2]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)[:, 1]
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)[:, 1]
    
    return X_meta_train, X_meta_test

def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test






Обучим один градиентный бустинг, в котором будем использовать 300 алгоритмов. Мы получим accuracy = 0.7899929527836504. Если мы будем агрегировать бустинг со случайным лесом и с двумя логистическими регрессиями, то получим результат лучше: 0.8118393234672304.



In [2]:
dataset = load_digits()
X, y = dataset['data'], dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=False, random_state=42, test_size=0.2)

In [4]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """    Эта функция подсчитывает признаки для мета-классификатора.    
    Они являются вероятностями классов при решении задачи многоклассовой классификации. 
    :arg clf: классификатор    :args X_train, y_train: обучающая выборка   
    :arg X_test: признаки тестовой выборки    :arg cv: класс, генерирующий фолды (KFold)    
    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(
            X_fold_predict)

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = meta_clf.predict_proba(X_test)

    return X_meta_train, X_meta_test

Задание 6.6.1
1 point possible (graded)
В скринкасте мы разобрали схему генерации признаков в стекинге, когда для тестовой выборки алгоритм заново переобучался на всей тренировочной выборке. Реализуйте схему, когда вместо этого производится агрегация ответов всех обученных на фолдах классификаторов на тестовой выборке при помощи усреднения.

Логика решения:
1) Создадим X_meta_test, заполним его нулями (по аналогии с X_meta_train);
2) Далее на каждом шаге, где мы обучаем folded_clf.fit (X_fold_train, y_fold_train) и его предсказания на X_fold_predict запихиваем в X_meta_train[predict_fold_index] добавим еще одну строку, где в X_meta_test будем добавлять предсказания вероятностей folded_clf на X_test. Их можно сразу складывать друг с другом или сохранить много массивов, тогда в конце их нужно будет все сложить, а потом делить на количество сплитов (количество массивов равно количеству сплитов в кросс - валидации);
3) После цикла останется только усреднить все эти массивы, это и будет наш X_meta_test.
За основу нужно взять следующий код:

In [5]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора. 
    Они являются вероятностями классов при решении задачи многоклассовой классификации.

    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)

    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros(len(y_train),n_classes, dtype=float32)
    for  train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train = X_train[train_fold_index]
        X_fold_predict = X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]       
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
        X_meta_test += folded_clf.predict_proba(X_test)
        
    
    
    X_meta_test = X_meta_test/cv.get_n_splits()
    return X_meta_train, X_meta_test

In [8]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

In [6]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [4]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test





Задание 6.6.2
5 points possible (graded)
Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимое количество итераций — 2000
логистическая регрессия с L2-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — multinomial, максимальное допустимое количество итераций — 2000
случайный лес из 300 деревьев
градиентный бустинг из 200 деревьев
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [14]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

In [9]:
stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='saga',
                       max_iter=2000, multi_class='ovr', random_state=42),
    LogisticRegression(C=0.001, penalty='l2', solver='saga',
                       max_iter=2000, multi_class='ovr', random_state=42),
    RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42),
    GradientBoostingClassifier(n_estimators=200, random_state=42)
], X_train, X_test, y_train, cv)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [04:26<00:00, 66.70s/it]


In [10]:
clf = LogisticRegression(penalty='none',multi_class='auto' ,solver='lbfgs',random_state=42)
clf.fit(stacked_features_train, y_train)

LogisticRegression(penalty='none', random_state=42)

In [11]:
compute_metric(clf,stacked_features_train,y_train,stacked_features_test)

0.976219

Используйте функцию generate_meta_features для стекинга следующих алгоритмов: случайный лес из 300 деревьев случайный лес из 200 экстремальных деревьев Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'. Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [33]:
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42),
    RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)
], X_train, X_test, y_train, cv)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:17<00:00,  8.89s/it]


In [34]:
# clf = LogisticRegression(penalty='none', solver='lbfgs',random_state=42)
clf.fit(stacked_features_train, y_train)

LogisticRegression(penalty='none', random_state=42)

In [35]:
compute_metric(clf,stacked_features_train,y_train,stacked_features_test)

0.976633

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
метод ближайшего соседа (k-NN) со стандартными параметрами
случайный лес из 300 экстремальных деревьев
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [5]:
from sklearn.neighbors import KNeighborsClassifier

In [18]:
stacked_features_train, stacked_features_test = generate_meta_features([
    KNeighborsClassifier(),
    RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)
], X_train, X_test, y_train, cv)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.61s/it]


In [19]:
clf = LogisticRegression(penalty='none', solver='lbfgs',multi_class='auto',random_state=42)
clf.fit(stacked_features_train, y_train)

C:\Users\vetak\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(penalty='none', random_state=42)

In [20]:
compute_metric(clf,stacked_features_train,y_train,stacked_features_test)

C:\Users\vetak\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.985552

Используйте функцию generate_meta_features для стекинга следующих алгоритмов: логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимоей количество итераций — 2000 метод ближайшего соседа со стандартными параметрами случайный лес из 300 экстремальных деревьев AdaBoost со стандартными параметрами Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'. Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

3
Используйте функцию generate_meta_features для стекинга следующих алгоритмов: метод ближайшего соседа (k-NN) со стандартными параметрами случайный лес из 300 экстремальных деревьев Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'. Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [12]:
stacked_features_train, stacked_features_test = generate_meta_features([
    KNeighborsClassifier(),
    RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)
], X_train, X_test, y_train, cv)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.20s/it]


In [13]:

clf = LogisticRegression(penalty='none', solver='lbfgs',multi_class='auto',random_state=42)
clf.fit(stacked_features_train, y_train)

LogisticRegression(penalty='none', random_state=42)

In [14]:
compute_metric(clf,stacked_features_train,y_train,stacked_features_test)

0.985552

4
Используйте функцию generate_meta_features для стекинга следующих алгоритмов: логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимоей количество итераций — 2000 метод ближайшего соседа со стандартными параметрами случайный лес из 300 экстремальных деревьев AdaBoost со стандартными параметрами Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'. Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [15]:
stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='saga',
                       max_iter=2000, multi_class='ovr', random_state=42),
    KNeighborsClassifier(),
    RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42),
    AdaBoostClassifier(random_state=42)
], X_train, X_test, y_train, cv)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:19<00:00, 19.77s/it]


In [16]:
clf = LogisticRegression(penalty='none', solver='lbfgs',random_state=42)
clf.fit(stacked_features_train, y_train)

LogisticRegression(penalty='none', random_state=42)

In [17]:
compute_metric(clf,stacked_features_train,y_train,stacked_features_test)

0.983693

In [23]:
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42),
    RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)
], X_train, X_test, y_train,cv)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:21<00:00, 10.84s/it]


In [24]:
clf = LogisticRegression(penalty='none', solver='lbfgs',random_state=42)
clf.fit(stacked_features_train, y_train)

LogisticRegression(penalty='none', random_state=42)

In [25]:
compute_metric(clf,stacked_features_train,y_train,stacked_features_test)


0.976633

In [52]:
cv = KFold(n_splits=20, shuffle=True, random_state=42)

В предыдущей задаче измените 10 фолдов на 20. Укажите полученное качество.

In [53]:
stacked_featuers_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42),
    ExtraTreesClassifier(n_estimators=200, n_jobs=-1, random_state=42)
    
], X_train, X_test, y_train,cv)






  0%|                                                                                            | 0/2 [02:24<?, ?it/s]





 50%|██████████████████████████████████████████                                          | 1/2 [00:21<00:21, 21.86s/it]




100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:34<00:00, 17.25s/it]


In [54]:
clf = LogisticRegression(penalty='none', solver='lbfgs',random_state=42)
clf.fit(stacked_features_train, y_train)
compute_metric(clf,stacked_features_train,y_train,stacked_features_test)

0.97531

В предыдущей задаче укажите количество фолдов равным 5 и поменяйте мета-алгоритм на случайный лес со стандартными параметрами. Укажите полученное качество.

In [38]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [39]:
clf = RandomForestClassifier(random_state=42)
clf.fit(stacked_features_train, y_train)
compute_metric(clf,stacked_features_train,y_train,stacked_features_test)

0.981669

В предыдущей задаче поменяйте мета-алгоритм на метод ближайших соседей (k-NN) со стандартными параметрами. Укажите полученное качество.

In [40]:
clf = KNeighborsClassifier()
clf.fit(stacked_features_train, y_train)
compute_metric(clf,stacked_features_train,y_train,stacked_features_test)

0.98762

В предыдущей задаче поменяйте мета-алгоритм на градиентный бустинг со стандартными параметрами. Укажите полученное качество.

In [41]:
clf = GradientBoostingClassifier()
clf.fit(stacked_features_train, y_train)
compute_metric(clf,stacked_features_train,y_train,stacked_features_test)

0.983607

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
случайный лес из 300 экстремальных деревьев
Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).
Для генерации фолдов используйте класс StratifiedKFold и поправленный Вами ранее код в функции compute_meta_feature.
Выполните разбиение на 3 фолда.
Как мета-алгортм используйте случайный лес из 100 экстремальных деревьев. Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [8]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):

    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]

    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])

    return stacked_features_train, stacked_features_test

In [10]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [12]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300,criterion= 'gini', n_jobs=-1,max_depth=24, random_state=42),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42)
], X_train, X_test, y_train,skf)

clf =  RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
clf.fit(stacked_features_train, y_train)

compute_metric(clf,stacked_features_train,y_train,stacked_features_test)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.51s/it]


0.932859

In [19]:
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300,criterion= 'gini', n_jobs=-1,max_depth=24, random_state=42),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42),
   
    
], X_train, X_test, y_train,skf)



100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.54s/it]


In [20]:
def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

In [21]:
clf =  ExtraTreesClassifier(n_estimators=100, n_jobs=-1, random_state=42)
clf.fit(stacked_features_train, y_train)

compute_metric(clf,stacked_features_train,y_train,stacked_features_test)

0.932859

In [22]:
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300,criterion= 'gini', n_jobs=-1,max_depth=24, random_state=42),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42),
    LogisticRegression()
    
], X_train, X_test, y_train,skf)


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:09<00:00,  3.06s/it]


In [23]:
clf =  ExtraTreesClassifier(n_estimators=100, n_jobs=-1, random_state=42)
clf.fit(stacked_features_train, y_train)

compute_metric(clf,stacked_features_train,y_train,stacked_features_test)

0.924933

In [25]:
np.sum(clf.predict_proba(stacked_features_test).argmax(axis=1))

1677